## Process dms data aggregation
This notebook is for processing dms data aggregation from MaveDB

In [1]:
import sqlite3
import csv
import math
import os
import urllib.request
import pandas as pd
import numpy as np
import shutil
import logging
import requests
from datetime import datetime
from Bio.Align import substitution_matrices
from Bio import AlignIO
from collections import Counter
from collections import defaultdict
from typing import Dict

In [2]:
# Setup logging function
def setup_logging(function_name):
    log_dir = 'logs'
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f'{function_name}.log')

    logging.basicConfig(
        filename=log_file,
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

In [3]:
# CONNECT to the database
conn = sqlite3.connect('compar_gen_aggregated_data.db')
cursor = conn.cursor()

### Dataset
focus of this analysis is amino acid substitutions compared with substitution scores from matrices like BLOSUM62, PAM250 ...
#### Datasets inclusion criteria
If csv scoreset from MaveDB does not meet all of the following criteria, it's excluded
| Inclusion criteria | Reason |
| ------------------ | --------------- |
| hgvs_pro value is not 'NA', i.e SNP | SNPs represent changes at the DNA level (single nucleotide changes), which do not always lead to a change in the amino acid sequence. Including SNPs that don't result in amino acid changes would introduce noise into your analysis, as these would not be relevant for a comparison with amino acid substitution scores. |
| mutation type = substitution, synonimous, nonsense | Substitution matrices like BLOSUM62 are designed to evaluate the evolutionary likelihood and functional impact of amino acid substitutions. Other types of mutations (e.g., insertions, deletions, frameshifts) alter the protein in different ways and are not comparable to simple substitutions. Including non-substitution mutations in the analysis would lead to mismatched comparisons, as the matrices do not provide relevant scores for these types of mutations. |
| single mutant | Double or multiple mutants introduce combinatorial effects that are more complex and not directly comparable to the single substitution scores in BLOSUM62. Including multiple mutants would complicate the analysis and make it difficult to draw clear conclusions about the relationship between DMS scores and BLOSUM62 scores. |
| species=Homo sapience | The lack of sufficient data for non-human species in MaveDB, particularly vertebrates, means that any comparative analysis might be statistically underpowered or biased. When more data becomes available, the analysis will be more credible. |
| 'score' in dms file is not empty |  |
| 'position' in dms file is not empty |  |
| unique amino acid mutation | exclude redundant nucleotide mutations that result in the same amino acid change. E.g., c.178_180delinsTAG and c.178_180delinsTAA (in the score set urn:mavedb:00000001-b-2) lead to the same amino acid change, p.Asn60Ter, the scores in these cases are the same, i.e. duplicates |

#### Data pre-processing
csv files with dms scores downloaded from MaveDB https://zenodo.org/records/11201737 need to be pre-processed before propagating data from them to msa-dms-db database. 
1. Data in this notebook has been already processed with workflow in Galaxy https://usegalaxy.eu/u/polina/w/dms-aggregated-parse-hgvs-extract-positions
   1. parse hgvs_pro to position, ancestral_residue, variant_residue
   2. join parsed table with original table from mavedb by hgvs_pro column

3. Convert three-code amino acid to one-code (preprocess_all_csvs_convert_3to1letter)

#### Data processing
1. Populate data to compar_gen_aggregated_data.db (fill_tables_with_all_dms_csv_in_folder)
   1. gene and species are fetched from MaveDB API https://api.mavedb.org/api/v1/score-sets/{urn_mavedb} and populated to db
    - Challenges: requires more computational and memory resources (applied for VM with more resources)
1. Calculate minmax scaled dms scores and fill in db (calculate_minmax_scaled_scores)
    - Challenges: currently produces duplicates, need to debug
   

In [ ]:
#FUNCTIONS 
# Data preprocessing (before filling tables)

# Define the dictionary to map three-letter to one-letter amino acid codes
AA_CODES: Dict[str, str] = {
    "Ala": "A",  # Alanine
    "Arg": "R",  # Arginine
    "Asn": "N",  # Asparagine
    "Asp": "D",  # Aspartic acid (Aspartate)
    "Cys": "C",  # Cysteine
    "Gln": "Q",  # Glutamine
    "Glu": "E",  # Glutamic acid (Glutamate)
    "Gly": "G",  # Glycine
    "His": "H",  # Histidine
    "Ile": "I",  # Isoleucine
    "Leu": "L",  # Leucine
    "Lys": "K",  # Lysine
    "Met": "M",  # Methionine
    "Phe": "F",  # Phenylalanine
    "Pro": "P",  # Proline
    "Ser": "S",  # Serine
    "Thr": "T",  # Threonine
    "Trp": "W",  # Tryptophan
    "Tyr": "Y",  # Tyrosine
    "Val": "V",  # Valine
    "Ter": "*",  # Termination codon
}


def preprocess_csv_convert_3to1letter(file_path: str, output_folder: str):
    """Process a single CSV file to map amino acid codes from three-letter to one-letter notation 
    creates an output file with extra columns: 'wt_1letter' in output file corresponds to 'ancestral' in input file,
    while 'variant_1letter' in output corresponds to 'variant' in input
    and save the output."""
    try:
        # Read the CSV file with low_memory=False to avoid DtypeWarning
        df = pd.read_csv(file_path, low_memory=False)

        # Map three-letter codes to one-letter codes for ancestral
        df['wt_1letter'] = df['ancestral'].map(AA_CODES)

        # Map three-letter codes to one-letter codes for variant
        df['variant_1letter'] = df['variant'].map(AA_CODES)

        # If variant is '0', replace it with the one-letter code from ancestral
        df['variant_1letter'] = df.apply(
            lambda row: row['wt_1letter'] if row['variant'] == '0' else row['variant_1letter'], axis=1)

        # Save the processed CSV
        output_file = os.path.join(output_folder, os.path.basename(file_path))
        df.to_csv(output_file, index=False)
    
    except Exception as e:
        # Print the file name and the error message if an exception occurs
        print(f"Issue occurred with file: {file_path}. Error: {str(e)}")

def preprocess_all_csvs_convert_3to1letter(input_folder: str, output_folder: str):
    """Process all CSV files with preprocess_csv_convert_3to1letter in the input folder and save them to the output folder."""
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each CSV file in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_folder, file_name)
            preprocess_csv_convert_3to1letter(file_path, output_folder)

    print("Processing completed.")

def move_preprocessed_files(all_in, preprocessed_out, preprocessed_in):
    """Helping function.
    Checks which files were processed with process_all_scvs and moves all 
    processed to folder 'preprocessed_in' from 'all_in'."""
    # Ensure the output directories exist
    os.makedirs(preprocessed_in, exist_ok=True)

    # Get the list of files in all_in and preprocessed_out
    all_in_files = set(os.listdir(all_in))
    preprocessed_out_files = set(os.listdir(preprocessed_out))

    # Find common files between all_in and preprocessed_out
    common_files = all_in_files.intersection(preprocessed_out_files)

    # Move common files from all_in to preprocessed_in
    for file_name in common_files:
        source_path = os.path.join(all_in, file_name)
        destination_path = os.path.join(preprocessed_in, file_name)
        shutil.move(source_path, destination_path)
        #print(f"Moved: {file_name} from {all_in} to {preprocessed_in}")

In [ ]:
preprocess_all_csvs_convert_3to1letter('data/mavedb-scores', 'data/mavedb-scores-1letter')

In [ ]:
move_preprocessed_files('data/mavedb-scores', 'data/mavedb-scores-1letter', 'data/mavedb-scores-preprocessed')

In [4]:
# Function to get target gene and organism name from MAVEdb
def get_target_gene_and_organism(urn_mavedb):
    api_url = f"https://api.mavedb.org/api/v1/score-sets/{urn_mavedb}"
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        target_gene = data['targetGenes'][0]['name']
        target_organism_name = data['targetGenes'][0]['targetSequence']['taxonomy']['organismName']
        return target_gene, target_organism_name
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from API for URN {urn_mavedb}: {e}")
        return None, None

# Function to fill tables with aggregated DMS data
def fill_tables_with_dms_csv(file_path, urn_mavedb, target_gene, target_organism_name):
    setup_logging('fill_tables_with_dms_csv')

    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            if row['type'] not in ['substitution', 'synonymous']:
                logging.info(f"Skipping row due to 'type': {row['type']}")
                continue

            if row['hgvs_pro'].startswith('p.['):
                logging.info(f"Skipping row due to 'hgvs_pro': {row['hgvs_pro']}")
                continue

            try:
                score = float(row['score']) if row['score'].strip() else None
            except ValueError as e:
                logging.error(f"Error converting score to float for row: {row}. Error: {e}")
                continue
            
            try:
                start = int(row['start'])
            except ValueError as e:
                logging.error(f"Error converting start to int for row: {row}. Error: {e}")
                continue
            
            wt_1letter = row['wt_1letter']
            variant_1letter = row['variant_1letter']

            # Insert gene into Gene table with URN
            cursor.execute('SELECT gene_id FROM Gene WHERE gene_name = ?', (target_gene,))
            result = cursor.fetchone()
            if result:
                gene_id = result[0]
                cursor.execute('UPDATE Gene SET urn_mavedb = ? WHERE gene_id = ?', (urn_mavedb, gene_id))
            else:
                cursor.execute('INSERT INTO Gene (gene_name, urn_mavedb) VALUES (?, ?)', (target_gene, urn_mavedb))
                gene_id = cursor.lastrowid

            # Insert species into Species table
            cursor.execute('SELECT species_id FROM Species WHERE species_name = ?', (target_organism_name,))
            result = cursor.fetchone()
            if result:
                species_id = result[0]
            else:
                cursor.execute('INSERT INTO Species (species_name) VALUES (?)', (target_organism_name,))
                species_id = cursor.lastrowid

            # Check if the mutation already exists in the Mutation table
            cursor.execute('''
                SELECT mutation_id FROM Mutation 
                WHERE gene_id = ? AND species_id = ? AND position = ? AND ancestral_residue = ? AND variant_residue = ?
            ''', (gene_id, species_id, start, wt_1letter, variant_1letter))
            result = cursor.fetchone()
            if result:
                mutation_id = result[0]
            else:
                cursor.execute('''
                    INSERT INTO Mutation (gene_id, species_id, position, ancestral_residue, variant_residue)
                    VALUES (?, ?, ?, ?, ?)
                ''', (gene_id, species_id, start, wt_1letter, variant_1letter))
                mutation_id = cursor.lastrowid

            # Insert DMS data into the DMS table
            if score is not None:
                cursor.execute('''
                    INSERT INTO DMS (mutation_id, score)
                    VALUES (?, ?)
                ''', (mutation_id, score))
            else:
                logging.info(f"Skipping row with invalid score: {row}")    
    conn.commit()

# Function to fill tables with all DMS CSV files in a folder
def fill_tables_with_all_dms_csv_in_folder(folder_path):
    setup_logging('fill_tables_with_all_dms_csv_in_folder')

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            # Extract urn_mavedb from the filename
            urn_mavedb = filename.replace('csv_urn-mavedb-', 'urn:mavedb:').replace('.scores.csv', '')

            # Get target gene and organism name from API
            target_gene, target_organism_name = get_target_gene_and_organism(urn_mavedb)
            if target_organism_name != 'Homo sapiens':
                logging.info(f"Skipping file due to organism name: {target_organism_name}")
                continue

            file_path = os.path.join(folder_path, filename)
            logging.info(f"Processing file: {file_path} with target_gene: {target_gene}")
            fill_tables_with_dms_csv(file_path, urn_mavedb, target_gene, target_organism_name)

In [7]:
# process all files in the folder
fill_tables_with_all_dms_csv_in_folder('data/batches/batch13')

In [8]:
#Script is producing duplicated, fix it
#FUNC
#use calculate_minmax_scaled_scores with species_name='Homo sapiens' for aggregated datasets

def calculate_minmax_scaled_scores(table_name, score_field, species_name=None, neutral_data_point=0):
    setup_logging('calculate_minmax_scaled_scores')
    logging.info(f"Processing table: {table_name} with species_name: {species_name} and neutral_data_point: {neutral_data_point}")
    # Extract data from the specified table
    if table_name == 'DMS':
        query = '''
            SELECT M.gene_id, M.mutation_id, D.{score_field}
            FROM DMS D
            JOIN Mutation M ON D.mutation_id = M.mutation_id
            JOIN Gene G ON M.gene_id = G.gene_id
            JOIN Species S ON M.species_id = S.species_id
            WHERE S.species_name = ?
        '''
        query = query.format(score_field=score_field)
        df = pd.read_sql_query(query, conn, params=(species_name,))
        #print("DMS DataFrame Extracted:\n", df.head())  # Debugging print statement

    elif table_name == 'SubstitutionMatrices':
        query = f'''
            SELECT amino_acid_x, amino_acid_y, {score_field}
            FROM SubstitutionMatrices
            WHERE amino_acid_x != amino_acid_y
        '''
        df = pd.read_sql_query(query, conn)
        df['mutation_id'] = ''  # For consistency in processing
        #print("SubstitutionMatrices DataFrame Extracted:\n", df.head())  # Debugging print statement

    # Shift the scores by the neutral point
    df['shifted_score'] = df[score_field] - neutral_data_point
    #print("Shifted Scores:\n", df[['shifted_score']].head())  # Debugging print statement

    # Calculate the min and max of the shifted scores
    min_val = df['shifted_score'].min()
    max_val = df['shifted_score'].max()
    #print(f"Min Value: {min_val}, Max Value: {max_val}")  # Debugging print statement
    logging.info(f"Processing table {table_name} completed")

    # Apply custom normalization formula
    def custom_normalize(x):
        if x > 0:
            return x / max_val if max_val != 0 else 0
        elif x < 0:
            return x / abs(min_val) if min_val != 0 else 0
        elif x == 0:
            return 0
        else:
            return None

    df[f'{score_field}_minmax_scaled'] = df['shifted_score'].apply(custom_normalize)
    #print("Scaled Values:\n", df[[f'{score_field}_minmax_scaled']].head())  # Debugging print statement

    # Update the database with the scaled values
    if table_name == 'DMS':
        for mutation_id, scaled_value in zip(df['mutation_id'], df[f'{score_field}_minmax_scaled']):
            cursor.execute(f'''
                UPDATE DMS
                SET {score_field}_minmax_scaled = ?
                WHERE mutation_id = ?
            ''', (scaled_value, mutation_id))
    elif table_name == 'SubstitutionMatrices':
        for amino_acid_x, amino_acid_y, scaled_value in zip(df['amino_acid_x'], df['amino_acid_y'], df[f'{score_field}_minmax_scaled']):
            cursor.execute(f'''
                UPDATE SubstitutionMatrices
                SET {score_field}_minmax_scaled = ?
                WHERE amino_acid_x = ? AND amino_acid_y = ?
            ''', (scaled_value, amino_acid_x, amino_acid_y))

    conn.commit()


In [ ]:
calculate_minmax_scaled_scores('DMS', 'score', 'Homo sapiens')

In [12]:
# Close the connection
conn.close()

In [6]:
#working function to help to understand
#Fetch species and genes from mavedb api and write to the file
def fetch_genes_and_species(output_file):
    # Function to fetch target gene and organism name from the API
    def get_target_gene_and_organism(urn):
        api_url = f"https://api.mavedb.org/api/v1/score-sets/{urn}"
        
        try:
            response = requests.get(api_url)
            response.raise_for_status()
            data = response.json()
            
            target_gene = data['targetGenes'][0]['name']
            target_organism_name = data['targetGenes'][0]['targetSequence']['taxonomy']['organismName']
            
            return target_gene, target_organism_name
        
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None, None


    # Fetch the urn_mavedb values from the Gene table
    cursor.execute("SELECT gene_name FROM Gene")
    rows = cursor.fetchall()

    # Open the output file for writing
    with open(output_file, 'w') as file:
        # Iterate through the rows and fetch genes and species
        for row in rows:
            urn = row[0]
            target_gene, target_organism_name = get_target_gene_and_organism(urn)
            
            if target_gene and target_organism_name:
                # Write the gene and species to the file
                file.write(f"URN: {urn}\n")
                file.write(f"Gene: {target_gene}\n")
                file.write(f"Species: {target_organism_name}\n")
                file.write("\n")



    print(f"Gene and species information has been written to {output_file}.")

# Example usage
output_file = 'genes_and_species.txt'
fetch_genes_and_species(output_file)


An error occurred: 404 Client Error: Not Found for url: https://api.mavedb.org/api/v1/score-sets/PTEN
An error occurred: 404 Client Error: Not Found for url: https://api.mavedb.org/api/v1/score-sets/PAX6%20Homeobox%20domain
An error occurred: 404 Client Error: Not Found for url: https://api.mavedb.org/api/v1/score-sets/CYP2C19
Gene and species information has been written to genes_and_species.txt.
